## Attractor Landscape analysis

This is an example of how Boolean network model was analyzed in our paper.

We provide the codes for 'Attractor landscape analysis', 'Perturbation analysis', and 'Analysis of network dynamics' (Please refer to the Method section).


In [ ]:
import numpy as np
import pandas as pd
import itertools
import networkx as nx
import copy
import os
from collections import defaultdict, deque

from pyboolnet.file_exchange import bnet2primes, primes2bnet
from pyboolnet.interaction_graphs import primes2igraph
from pyboolnet.state_transition_graphs import primes2stg
from pyboolnet.attractors import compute_attractors_tarjan

from modules.attractorSim import rand_initial_states, compute_attractor_from_primes, compute_phenotype, Simulation
from modules.printlogic import print_node_logic

In [2]:
network_dir = './network/'
model_file = network_dir + 'EMT_Network.bnet'
primes = bnet2primes(model_file)
nodeList = list(primes.keys())
graph = primes2igraph(primes)
update_mode = "synchronous"

# nodes_order = sorted(primes.keys())  # 이미 선언된 상태여야 함
nodes_order = sorted(primes.keys())
print("총 노드 수:", len(nodes_order))
print("앞 31개 노드:", nodes_order[:31])

# (붙여넣기용) 자동 마커/phenotype 생성
# --- 위 2) 셀의 코드 전체를 여기에 붙여 넣으면 됨 ---

# --- 시작: phenotype/markers 자동 설정 (복붙해서 기존 phenotype 셀 대신 사용) ---
# nodes_order는 위에서 확인한 primes의 정렬된 노드 리스트

# 1) 마커 자동 찾기: E-cad (E-cadherin 계열)과 ZEB1 계열 노드 이름 검색
def find_node_by_keywords(nodes_order, keywords):
    for n in nodes_order:
        lower = n.lower()
        for kw in keywords:
            if kw.lower() in lower:
                return n
    return None

ecad_candidates = ["ecad", "cdh1", "e-cad", "ecadherin"]
zeb_candidates  = ["zeb1", "zeb"]

Ecad_node = find_node_by_keywords(nodes_order, ecad_candidates)
ZEB_node  = find_node_by_keywords(nodes_order, zeb_candidates)

if not Ecad_node or not ZEB_node:
    raise RuntimeError(f"E-cad 또는 ZEB1 노드를 찾을 수 없음. nodes 예시: {nodes_order[:30]}")

print("Detected markers -> E-cad:", Ecad_node, ", ZEB1:", ZEB_node)

# 2) markers 변수 (노트북의 기존 markers 변수 대체)
markers = [Ecad_node, ZEB_node]

# 3) phenotype 정의: 논문 기준 (Epithelial / Mesenchymal / Hybrid)
# P1..P4은 모든 조합(1/0) 중 논문에서 말한 4개 조합(원하면 이름 바꿔도 됨)
phenotype = {
    'P_epithelial': {Ecad_node:1, ZEB_node:0},   # epithelial (E-cad + / ZEB1 -)
    'P_mesenchymal': {Ecad_node:0, ZEB_node:1},  # mesenchymal (E-cad - / ZEB1 +)
    'P_hybrid_high': {Ecad_node:1, ZEB_node:1},  # hybrid (E-cad + / ZEB1 +)
    'P_hybrid_low' : {Ecad_node:0, ZEB_node:0}   # hybrid (E-cad - / ZEB1 -)
}

# 4) phenotypeAnnot: 숫자 레이블 관례 (논문/분석 용도에 맞게)
# 여기선 epithelial = -1, mesenchymal = +1, hybrid들 = 0 (논문/시각화에서 자주 사용하는 구분)
phenotypeAnnot = {
    'P_epithelial': -1,
    'P_mesenchymal': 1,
    'P_hybrid_high': 0,
    'P_hybrid_low': 0
}

print("phenotype keys:", list(phenotype.keys()))
print("markers:", markers)
# --- 끝 ---


if 2**len(nodeList) >= 100000: num_init = 100000
else:  num_init = 2**len(nodeList)
initState = rand_initial_states(num_init, len(nodeList))

총 노드 수: 31
앞 31개 노드: ['AKT', 'AP1', 'ERK', 'Ecadherin', 'EpCAM', 'GLI', 'GRB2SOS', 'GSK3b', 'IKK', 'MDM2', 'MEK', 'MYC', 'NFKB', 'NOTCH', 'PI3K', 'RAF', 'RAS', 'SMAD23', 'SMAD4', 'Snail', 'TAK1', 'TGFBR', 'TGFb', 'THY1', 'Twist1', 'ZEB1', 'bcatenin', 'miR200', 'miR34', 'p38', 'p53']
Detected markers -> E-cad: Ecadherin , ZEB1: ZEB1
phenotype keys: ['P_epithelial', 'P_mesenchymal', 'P_hybrid_high', 'P_hybrid_low']
markers: ['Ecadherin', 'ZEB1']


### Node perturbation analysis

In [3]:
save_dir = './result/EMT/' 
save_perturbname = save_dir + 'EMT_single_simul_result.csv'

In [4]:
fix_dict = {'RAS':1}
# allsingles = [{n:1} for n in nodeList] + [{n:0} for n in nodeList]

In [5]:
fix_dict_tmp = copy.deepcopy(fix_dict) # fix_dict는 {'RAS':1} 인 상태

print(fix_dict_tmp) # 현재 적용할 고정 노드 출력

primes_new, pheno_df, att_ave_pd, attrs_dict = Simulation(fix_dict_tmp, primes, update_mode, initState, phenotype, phenotypeAnnot)   

print(f'attrs_dict={attrs_dict}')


ori_attrs_num = len(attrs_dict)
print(f'어트랙터 개수: {ori_attrs_num}')


{'RAS': 1}
WARNING The state transition graph will consist of up to 2**31=2147483648 states, depending on the initial states.

--- [Simulation Function] Detected Attractors for Current Perturbation ---
  > Attractor ID 0 (Steady State, Basin 48.89%)
    State Str:  1110011011111111111111111110010
  > Attractor ID 1 (Cyclic, Length 4, Basin 1.40%)
    Cycle Start Str:  1001000101100001100000000001101
       Step 1 Str:  1001000101100001100000000001101
       Step 2 Str:  0001000101100001100000000001100
       Step 3 Str:  0001100100100011100000001000100
       Step 4 Str:  1001100100100011100000001000101
  > Attractor ID 2 (Cyclic, Length 7, Basin 49.72%)
    Cycle Start Str:  1101100001010010101000011110100
       Step 1 Str:  1101100001010010101000011110100
       Step 2 Str:  1001100001110011100000011110000
       Step 3 Str:  1001100000110011100000011110000
       Step 4 Str:  1001100000110011100000011110001
       Step 5 Str:  1011000001110001100000000111101
       Step 6 Str:  001

### 하나씩 없앤 네트워크 분석

In [6]:
def remove_char_at_index(s: str, index: int) -> str:
    """
    주어진 문자열에서 특정 인덱스의 문자를 제거합니다.
    """
    if not (0 <= index < len(s)):
        raise IndexError(f"인덱스 {index}가 문자열 길이 {len(s)}의 범위를 벗어납니다.")
    return s[:index] + s[index+1:]

def transform_attractor_for_comparison(attractor_tuple: tuple, removed_node_index: int) -> tuple:
    """
    original attractor 튜플의 각 문자열에서 removed_node_index에 해당하는 문자를 제거하여
    비교를 위한 새로운 튜플을 생성합니다.
    """
    transformed_strings = []
    for s in attractor_tuple:
        transformed_strings.append(remove_char_at_index(s, removed_node_index))
    return tuple(transformed_strings)

def are_cyclic_attractors_identical(attractor1, attractor2):
    """
    두 개의 cyclic attractor 튜플이 순환적으로 동일한지 비교합니다.
    """
    if len(attractor1) != len(attractor2):
        return False

    if not attractor1 or not attractor2: # 빈 튜플인 경우
        return False

    d1 = deque(attractor1)
    
    for _ in range(len(attractor1)):
        if tuple(d1) == attractor2:
            return True
        d1.rotate(1) # 한 칸씩 회전

    return False

def compare_attractors(attr_tuple1, attr_tuple2):
    """
    두 개의 attractor 튜플을 비교하여 동일한지 여부를 반환합니다.
    steady-state와 cyclic attractor를 구분하여 비교합니다.
    """
    len1 = len(attr_tuple1)
    len2 = len(attr_tuple2)

    if len1 == 1 and len2 == 1:
        return attr_tuple1[0] == attr_tuple2[0]
    elif len1 > 1 and len2 > 1:
        # ↓↓↓ 여기가 문제의 발생 원인이었습니다. attr_tuple1, attr_tuple2로 수정했습니다. ↓↓↓
        return are_cyclic_attractors_identical(attr_tuple1, attr_tuple2)
        # ↑↑↑
    else:
        return False

def find_identical_attractors(original_data, new_data, removed_node_index: int):
    """
    original 네트워크와 new 네트워크 간에 동일한 attractor를 찾아 출력합니다.
    original 네트워크의 각 attractor는 removed_node_index에 해당하는 노드를 제거한 후 비교합니다.
    """
    print(f"--- Attractor 비교 결과 (original에서 노드 {removed_node_index} 제거 후) ---")
    found_any_match = False

    for original_key, original_info in original_data.items():
        original_attractor_raw = original_info['attractors']
        
        try:
            if not original_attractor_raw:
                print(f"⚠️ Original {original_key}번 어트랙터 튜플이 비어있습니다. 건너뜁니다.")
                continue
            
            # 모든 문자열의 길이가 removed_node_index보다 길어야만 제거를 시도
            if any(len(s) <= removed_node_index for s in original_attractor_raw):
                 print(f"⚠️ Original {original_key}번 어트랙터 문자열 중 노드 {removed_node_index}를 제거할 수 없는 짧은 문자열이 있습니다. 건너뜁니다.")
                 continue

            original_attractor_transformed = transform_attractor_for_comparison(original_attractor_raw, removed_node_index)
            original_type = "Steady-state" if len(original_attractor_transformed) == 1 else "Cyclic"

        except IndexError as e:
            print(f"⚠️ Original {original_key}번 어트랙터 변형 중 오류 발생 (인덱스 {removed_node_index}): {e}. 이 어트랙터는 건너뜁니다.")
            continue
        
        match_found_for_this_original = False
        
        for new_key, new_info in new_data.items():
            new_attractor = new_info['attractors']
            new_type = "Steady-state" if len(new_attractor) == 1 else "Cyclic"

            if not original_attractor_transformed or not new_attractor:
                continue 
            
            # 변형된 original 어트랙터와 new 어트랙터의 문자열 길이가 다르면 비교하지 않음
            # 모든 문자열이 동일한 길이를 가지는 것을 가정
            if len(original_attractor_transformed[0]) != len(new_attractor[0]):
                continue

            if compare_attractors(original_attractor_transformed, new_attractor):
                print(f"✅ 동일한 attractor를 찾았습니다!")
                print(f"  - Original: {original_key}번 (원본: {original_attractor_raw}, {original_type})")
                print(f"    (노드 {removed_node_index} 제거 후): {original_attractor_transformed}")
                print(f"  - New     : {new_key}번 ({new_type})")
                print(f"    어트랙터: {new_attractor}")
                print("-" * 30)
                found_any_match = True
                match_found_for_this_original = True
                break
        
        if not match_found_for_this_original:
            print(f"❌ Original {original_key}번 어트랙터 (노드 {removed_node_index} 제거 후)는 New 어트랙터에서 일치하는 것을 찾지 못했습니다.")
            print(f"   (노드 제거 전): {original_attractor_raw}")
            print(f"   (노드 제거 후): {original_attractor_transformed}")
            print("-" * 30)

    if not found_any_match:
        print(f"💡 Original과 New 네트워크 사이에 (노드 {removed_node_index} 제거 후) 동일한 attractor를 찾을 수 없습니다.")

In [7]:
remove_network_dir = './network/biological_network/EMT_Network_'
remove_fix_dict = copy.deepcopy(fix_dict) # fix_dict는 {'RAS':1} 인 상태
remove_update_mode = "synchronous"

for i in range(len(nodes_order)):
    remove_node_now = nodes_order[i]
    print(f'------remove logic of {remove_node_now}------')
    try:
        remove_model_file = remove_network_dir + str(i) + '.bnet'
        remove_primes = bnet2primes(remove_model_file)
        print(remove_fix_dict) # 현재 적용할 고정 노드 출력

        remove_nodeList = list(remove_primes.keys())
        if 2**len(remove_nodeList) >= 100000: remove_num_init = 100000
        else:  remove_num_init = 2**len(remove_nodeList)
        remove_initState = rand_initial_states(remove_num_init, len(remove_nodeList))
        
        reomve_primes_new, remove_pheno_df, remove_att_ave_pd, remove_attrs_dict = Simulation(remove_fix_dict, remove_primes, remove_update_mode, remove_initState, phenotype, phenotypeAnnot)
        # print(f'\nremove_attrs_dict={remove_attrs_dict}\n')

        copy_ori_attrs = copy.deepcopy(attrs_dict)
        copy_remove_attrs = copy.deepcopy(remove_attrs_dict)
        # print(copy_ori_attrs)
        # print(copy_remove_attrs)
        # 기존: find_identical_attractors(copy_ori_attrs, remove_attrs_dict, i)
        try: # find_identical_attractors 내부에서 발생하는 예외를 더 상세히 잡기 위함
            find_identical_attractors(copy_ori_attrs, remove_attrs_dict, i)
        except Exception as e:
            print(f"DEBUG: find_identical_attractors 호출 중 예외 발생: {type(e).__name__}: {e}")
            raise # 이 예외를 다시 상위 try-except 블록으로 전달하여, 사용자 코드의 catch 문으로 가게 합니다.
        
        
    except:
        print("--현재 로직은 Phenotype Node 또는 가장 상위 Node 입니다.--")


------remove logic of AKT------
{'RAS': 1}
WARNING The state transition graph will consist of up to 2**30=1073741824 states, depending on the initial states.

--- [Simulation Function] Detected Attractors for Current Perturbation ---
  > Attractor ID 0 (Steady State, Basin 49.54%)
    State Str:  110011011111111111111111110010
  > Attractor ID 1 (Cyclic, Length 4, Basin 1.60%)
    Cycle Start Str:  001000101100001100000000001101
       Step 1 Str:  001000101100001100000000001101
       Step 2 Str:  001000101100001100000000001100
       Step 3 Str:  001100100100011100000001000100
       Step 4 Str:  001100100100011100000011000101
  > Attractor ID 2 (Cyclic, Length 7, Basin 48.86%)
    Cycle Start Str:  011000001110001100000000111101
       Step 1 Str:  011000001110001100000000111101
       Step 2 Str:  011000001100000101000001011100
       Step 3 Str:  111100001010010101000011010100
       Step 4 Str:  101100001010010101000011110100
       Step 5 Str:  001100001110011100000011110000
   